In [1]:
import sys
sys.path.append('../_amigocloud')

In [67]:
import collections
from amigocloud import AmigoCloud

from docxtpl import DocxTemplate
import docxtpl
from docx.shared import Mm

import qrcode

In [2]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

In [6]:
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [120]:
tipo_inf = 'INSPECCION'
fecha_actual = '08/01/2024'
anho = '2024'
cite = 1
tecnico = 'Juan Pablo Jaldin Duran'
fecha_insp = '08/01/2024'
canhero = 'MARIANO AGULERA'
cod_ca = 75

In [121]:
data = {
    "tipo_inf": tipo_inf,
    "fecha_actual": fecha_actual,
    "anho": anho,
    "cite": cite,
    "tecnico": tecnico,
    "fecha_insp": fecha_insp,
    "nom_ca": canhero,
    "cod_ca": cod_ca
}
data

{'tipo_inf': 'INSPECCION',
 'fecha_actual': '08/01/2024',
 'anho': '2024',
 'cite': 1,
 'tecnico': 'Juan Pablo Jaldin Duran',
 'fecha_insp': '08/01/2024',
 'nom_ca': 'MARIANO AGULERA',
 'cod_ca': 75}

In [122]:
data_inf = collections.namedtuple("data", data.keys())(*data.values())

In [123]:
data_inf

data(tipo_inf='INSPECCION', fecha_actual='08/01/2024', anho='2024', cite=1, tecnico='Juan Pablo Jaldin Duran', fecha_insp='08/01/2024', nom_ca='MARIANO AGULERA', cod_ca=75)

In [124]:
# GENERAR QR
texto_qr = '''
--UTEA--
--INFORME DE {tipo}--

Codigo de informe: {cite} - {anho}
Fecha de Inspeccion: {fecha_insp}
Nombre de Canero: {canero}
Codigo Canero: {cod_ca}
Tecnico Responsable: {tecnico}'''\
    .format(
        tipo=data_inf.tipo_inf,
        cite=str(data_inf.cite),
        anho = data_inf.anho,
        fecha_insp=data_inf.fecha_insp, 
        canero=data_inf.nom_ca,
        cod_ca=data_inf.cod_ca,
        tecnico=data_inf.tecnico)

In [125]:
qr = qrcode.QRCode(
        version=1,
        error_correction=qrcode.constants.ERROR_CORRECT_L,
        box_size=10,
        border=4)

In [126]:
qr.add_data(texto_qr)
qr.make(fit=True)    
qr_img = qr.make_image(fill_color="black", back_color="white")

In [127]:
qr_file_name = 'qr_' + str(data_inf.cod_ca) + '_' + data_inf.nom_ca + '_' + str(data_inf.cite) + '_' + data_inf.fecha_insp.replace('/', '-')
qr_file_name

'qr_75_MARIANO AGULERA_1_08-01-2024'

In [128]:
ruta_qr = ruta + '/planos/' + qr_file_name + '.png'
qr_img.save(ruta_qr)

In [137]:
# GENERAR REPORTE
doc = DocxTemplate(ruta + "/templates/tpl_modelo_informe.docx")

In [138]:
firma_respon = None
if data_inf.tecnico == 'Rogelio Acuña Rodríguez':
    firma_respon = docxtpl.InlineImage(doc, image_descriptor=ruta+'/templates/firma_rogelio.png', width=Mm(60))
else:
    firma_respon = docxtpl.InlineImage(doc, image_descriptor=ruta+'/templates/firma_jaldin.png', width=Mm(60))

In [139]:
qr_file = docxtpl.InlineImage(doc, image_descriptor=ruta_qr, width=Mm(30))

In [140]:
context = {'data':data_inf, 'firma':firma_respon,'qr':qr_file}
doc.render(context)

In [141]:
data_inf

data(tipo_inf='INSPECCION', fecha_actual='08/01/2024', anho='2024', cite=1, tecnico='Juan Pablo Jaldin Duran', fecha_insp='08/01/2024', nom_ca='MARIANO AGULERA', cod_ca=75)

In [142]:
file_name = "_" + str(data_inf.cod_ca) + '_INSP_' + data_inf.fecha_insp.replace('/', '-') + '_' + data_inf.nom_ca
file_name

'_75_INSP_08-01-2024_MARIANO AGULERA'

In [143]:
doc.save(ruta + '/informes/' + file_name + '.docx')